In [ ]:
params_file = None
output_file = None

# LSTM for sediment yield (K-fold)

In [ ]:
import glob
import json
import os
import numpy as np
import joblib
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from scipy.stats import pearsonr
from math import sqrt
import torch
from torch.functional import F
import torch.nn as nn
from ExlossUnivariate import ExlossUnivariate
from torch.utils.data import DataLoader, TensorDataset
import matplotlib.pyplot as plt
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# === LOAD PARAMETERS ===
with open(params_file, 'r') as g:
    f = json.load(g)
    # data parameters
    window_size = f['window_size']
    window_step = f['window_step']
    num_stations = f['num_stations']
    X_scaler_path = f['X_scaler_path']
    Y_scaler_path = f['Y_scaler_path']
    # model parameters
    lstm_hidden_size = f['lstm_hidden_size']
    linear_hidden_size = f['linear_hidden_size']
    lstm_num_layers = f['lstm_num_layers']
    lstm_dropout= f['lstm_dropout']
    linear_dropout= f['linear_dropout']
    if_layer_norm = f['if_layer_norm']
    if_gru = f['if_gru']
    # training parameters
    batch_size = f['batch_size']
    num_epochs = f['num_epochs']
    lr = f['learning_rate']
    up_th = f['up_th']
    down_th = f['down_th']
    lambda_underestimate = f['lambda_underestimate']
    lambda_overestimate = f['lambda_overestimate']
    lambda_init = f['lambda_init']

file_output = output_file

In [ ]:
scaler_filename = X_scaler_path
X_loaded_scaler = joblib.load(scaler_filename)
scaler_filename = Y_scaler_path
Y_loaded_scaler = joblib.load(scaler_filename)

In [ ]:
# transform_func = np.exp
def transform_func(x):
    return 10 ** x

In [ ]:
# ==== Define Model ====
class CustomLSTM(nn.Module):
    def __init__(self, 
                 input_size, 
                 lstm_hidden_size,
                 linear_hidden_size,
                 lstm_num_layers=2,
                 lstm_dropout=0.2,
                 linear_dropout=0.5,
                 if_layer_norm=True,
                 if_gru=False,
                ):
        super().__init__()
        if if_gru:
            if lstm_num_layers > 1:
                self.lstm = nn.GRU(input_size, 
                                    lstm_hidden_size, 
                                    lstm_num_layers, 
                                    batch_first=True,
                                    dropout=lstm_dropout,
                                )
            else:
                self.lstm = nn.GRU(input_size, 
                                    lstm_hidden_size, 
                                    1, 
                                    batch_first=True,
                                    dropout=0.0,
                                )
        else:
            if lstm_num_layers > 1:
                self.lstm = nn.LSTM(input_size, 
                                    lstm_hidden_size, 
                                    lstm_num_layers, 
                                    batch_first=True,
                                    dropout=lstm_dropout,
                                )
            else:
                self.lstm = nn.LSTM(input_size, 
                                    lstm_hidden_size, 
                                    1, 
                                    batch_first=True,
                                    dropout=0.0,
                            )
        self.layer_norm = nn.LayerNorm(lstm_hidden_size)
        self.fc1 = nn.Linear(linear_hidden_size, linear_hidden_size)
        self.dropout = nn.Dropout(p=linear_dropout)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(linear_hidden_size, 1)
        self.lstm_hidden_size = lstm_hidden_size
        self.if_ = if_layer_norm

    def forward(self, x):
        # access dimension data
        # x: (batch, seq_len, input_size)
        batch_size, sequence_length, num_features = x.shape
        # all timesteps
        out, _ = self.lstm(x)
        # consider layer normalization
        if self.if_:
            out = self.layer_norm(out)
        # flatten and linear layers
        out = self.fc1(out.contiguous().view(-1, self.lstm_hidden_size))
        out = self.relu(out)
        out = self.dropout(out)
        out = self.fc2(out)
        out = out.view(batch_size, sequence_length, -1)
        out = out.squeeze(-1)
        return out

In [ ]:
def kge(Y, Yhat):
    '''
    Compute Kling-Gupta efficiency

    Parameters
    ----------
    Y : true values vector
    Yhat : predicted values vector

    Return
    ------
    float
    '''
    R = float(pearsonr(Y, Yhat).statistic[0])
    Y_std = np.std(Y)
    Yhat_std = np.std(Yhat)
    Y_mu = np.mean(Y)
    Yhat_mu = np.mean(Yhat)
    beta = float(Yhat_mu / Y_mu)
    alpha = float(Yhat_std / Y_std)
    kappa = 1 - sqrt((R-1)**2 + (beta - 1)**2 + (alpha - 1)**2)
    return kappa, beta, alpha, R

In [3]:
def init_lstm_weights(m):
    if isinstance(m, torch.nn.LSTM):
        for name, param in m.named_parameters():
            if 'weight_ih' in name:
                torch.nn.init.xavier_uniform_(param.data)
            elif 'weight_hh' in name:
                torch.nn.init.orthogonal_(param.data)

In [ ]:
# ==== Load Data ====

# Change to your target directory if needed, or specify the full path
search_directory = "training" 

# Use '*' as a wildcard for the suffix
pattern = os.path.join(search_directory, f"X-size{window_size}-step{window_step}-station{num_stations}_*.npy")

# Find all files matching the pattern
files_found = glob.glob(pattern)

print("Files found using glob:")
years = []
for file_path in files_found:
    print(file_path)
    years += [file_path[file_path.find('_')+1:-4].strip('_').split('_')]

In [ ]:

with open(file_output,'w') as f:

    f.write('Model parameters:\n')
    f.write('-----\n')
    f.write(f'lstm_hidden_size:{lstm_hidden_size}\n')
    f.write(f'linear_hidden_size:{linear_hidden_size}\n')
    f.write(f'lstm_num_layers:{lstm_num_layers}\n')
    f.write(f'lstm_dropout:{lstm_dropout}\n')
    f.write(f'linear_dropout:{linear_dropout}\n')
    f.write(f'if_layer_norm:{if_layer_norm}\n')
    f.write(f'if_gru:{if_gru}\n')
    f.write('\n')

    f.write('Training parameters:\n')
    f.write('-----\n')
    f.write(f'batch_size:{batch_size}\n')
    f.write(f'num_epochs:{num_epochs}\n')
    f.write(f'learning_rate:{lr}\n')
    f.write(f'up_th:{up_th}\n') 
    f.write(f'down_th:{down_th}\n')
    f.write(f'lambda_underestimate:{lambda_underestimate}\n')
    f.write(f'lambda_overestimate:{lambda_overestimate}\n')
    f.write(f'lambda_init:{lambda_init}\n')
    f.write('\n')

    # data parameters
    f.write('Data parameters:\n')
    f.write('-----\n')
    f.write(f'window_size:{window_size}\n')
    f.write(f'window_step:{window_step}\n')
    f.write(f'num_stations:{num_stations}\n')
    f.write('\n')

    suffixes = []
    for file_path in files_found:
        suffix = '_' + ''.join(file_path[file_path.find('_')+1:-4])
        suffixes.append(suffix)

    K = len(suffixes)
    for k in range(K):

        # ==== k fold validation ====
        suffix = suffixes[k]
        X_val = np.load(f"training/X-size{window_size}-step{window_step}-station{num_stations}{suffix}.npy")
        Y_val = np.load(f"training/Y-size{window_size}-step{window_step}-station{num_stations}{suffix}.npy")
        X_val = torch.tensor(X_val, dtype=torch.float32)
        Y_val = torch.tensor(Y_val, dtype=torch.float32)

        # ==== k fold training ====
        suffix_not = [suffixes[_] for _ in range(K) if _ == k]
        suffix_n = suffix_not[0]
        X_train = np.load(f"training/X-size{window_size}-step{window_step}-station{num_stations}{suffix_n}.npy")
        Y_train = np.load(f"training/Y-size{window_size}-step{window_step}-station{num_stations}{suffix_n}.npy")
        for suffix_n in suffix_not:
            X_next = np.load(f"training/X-size{window_size}-step{window_step}-station{num_stations}{suffix_n}.npy")
            Y_next = np.load(f"training/Y-size{window_size}-step{window_step}-station{num_stations}{suffix_n}.npy")
            X_train = np.vstack((X_train,X_next))
            Y_train = np.concatenate((Y_train,Y_next))
        X_train = torch.tensor(X_train, dtype=torch.float32)
        Y_train = torch.tensor(Y_train, dtype=torch.float32)

        # ==== Wrap into a dataset and dataloader ====
        dataset_train = TensorDataset(X_train, Y_train)
        dataloader_train = DataLoader(dataset_train, batch_size=batch_size, shuffle=True) # must shuffle
        dataset_val = TensorDataset(X_val, Y_val)
        dataloader_val = DataLoader(dataset_val, batch_size=batch_size, shuffle=False)

        # ==== Determine model dimensions from data ====
        input_size = X_next.shape[2]
        output_size = Y_next.shape[1] if Y_next.ndim == 2 else Y_next.shape[2]

        # ==== Initialize Model, Loss Function, Optimizer ====
        model = CustomLSTM(input_size, 
                        lstm_hidden_size=lstm_hidden_size,
                        linear_hidden_size=linear_hidden_size,
                        lstm_num_layers=lstm_num_layers,
                        lstm_dropout=lstm_dropout,
                        linear_dropout=linear_dropout,
                        if_layer_norm=if_layer_norm,
                        if_gru=if_gru,
                        )
        model = model.to(device)
        criterion = ExlossUnivariate
        optimizer = torch.optim.Adam(model.parameters(), lr=lr)
        if_initialize = True
        if if_initialize:
            model.apply(init_lstm_weights)

        f.write(f'KFold:{suffix}\n')
        f.write('\n')
        print(f'KFold:{suffix}\n')
        print()

        # ==== Training Loop ====

        for epoch in range(num_epochs):

            ### fitting ###

            # training
            model.train()
            for xb, yb in dataloader_train:
                xb, yb = xb.to(device), yb.to(device)
                optimizer.zero_grad()
                preds = model(xb)
                loss = criterion(preds, yb,
                                up_th,
                                down_th,
                                lambda_underestimate,
                                lambda_overestimate,
                                lambda_init
                                )
                loss.backward()
                optimizer.step()

            ### evaluating ###

            # validation
            model.eval()
            running_loss_val = 0.0
            for xb, yb in dataloader_val:
                xb, yb = xb.to(device), yb.to(device)
                preds = model(xb)
                loss = criterion(preds, yb)
                running_loss_val += loss.item() * xb.size(0)
            epoch_loss_val = running_loss_val / len(dataset_val)

            # training
            model.eval()
            running_loss_train = 0.0
            for xb, yb in dataloader_train:
                xb, yb = xb.to(device), yb.to(device)
                preds = model(xb)
                loss = criterion(preds, yb)
                running_loss_train += loss.item() * xb.size(0)
            epoch_loss_train = running_loss_train / len(dataset_train)

            ### compute kge ###
            model.eval()
            # put on gpu
            Xg = X_val.to(device)
            Yg = Y_val.to(device)
            Yhat = model(Xg)
            Yg = Yg.detach().cpu()
            Yhat = Yhat.detach().cpu()
            Yg = Yg.numpy()
            Yhat = Yhat.numpy()
            Yg_reshaped = Yg.reshape(-1,1)
            Yhat_reshaped = Yhat.reshape(-1,1)
            Yg_unscaled = Y_loaded_scaler.inverse_transform(Yg_reshaped)
            Yhat_unscaled = Y_loaded_scaler.inverse_transform(Yhat_reshaped)
            Yg_undid = transform_func(Yg_unscaled)
            Yhat_undid = transform_func(Yhat_unscaled)
            kappa1, beta1, alpha1, R1 = kge(Yg_unscaled, Yhat_unscaled)
            kappa2, beta2, alpha2, R2 = kge(Yg_undid, Yhat_undid)

            # Print losses over time
            print(f"Epoch {epoch + 1}/{num_epochs}, TrainLoss:{epoch_loss_train:.4f}, ValLoss:{epoch_loss_val:.4f}, ValKGE(orig):{kappa2:.4f}")
            f.write(f"Epoch{epoch + 1}/{num_epochs},TrainLoss:{epoch_loss_train:.4f},ValLoss:{epoch_loss_val:.4f},ValKGE(orig):{kappa2:.4f},ValBeta(orig):{beta2:.4f},ValAlpha(orig):{alpha2:.4f},ValPearson(orig):{R2:.4f}\n")
        f.write('\n')
        print()

        

Files found using glob:
training/X-size60-step20-station50_2000_2004_2020.npy
training/X-size60-step20-station50_1999_2009_2015.npy
training/X-size60-step20-station50_1994_1998_2007.npy
training/X-size60-step20-station50_1992_2002_2012.npy
training/X-size60-step20-station50_2003_2005_2014.npy
training/X-size60-step20-station50_1993_2008_2018.npy
training/X-size60-step20-station50_1995_1997_2010.npy
training/X-size60-step20-station50_2013_2017_2019.npy
